In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


# Characteristics of the dataset

In [4]:
print ("The number of interactions is {}".format(len(URM_all_dataframe)))

The number of interactions is 1764607


In [5]:
userID_unique = URM_all_dataframe["user_id"].unique()
itemID_unique = URM_all_dataframe["item_id"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 38121, Number of users	 35736
Max ID items	 38120, Max Id users	 35735

Average interactions per user 49.38
Average interactions per item 46.29

Sparsity 99.87 %


# Splitting dataset

In [6]:
import scipy.sparse as sps

URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))


## K-fold Cross-validation
To find the optimal parameters, we should use cross-validation to better find our parameters

In [7]:
k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]
assert len(seeds) == k_fold

In [ ]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()
urm_trains = []
urm_validations = []
urm_test = []
# Defining evaluators
evaluator_validation = []
evaluator_test = []

for i in range(k_fold):
    # Splitting data into 64% train, 16% validation, 20% test
    URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=seeds[i])
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8, seed=seeds[i])
    
    urm_trains.append(URM_train)
    urm_validations.append(URM_validation)
    urm_test.append(URM_test)
    
    evaluator_validation.append(EvaluatorHoldout(URM_validation, cutoff_list=[10]))
    evaluator_test.append(EvaluatorHoldout(URM_test, cutoff_list=[10]))

EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 447 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 155 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 441 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 467 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 169 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 413 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 154 ( 0.4%) Users that have less than 

# Optuna: finding the optimal hyperparameters

### Step 2: Define the objective function that will be run by the optimizer

The obective function should:
* Use the optuna trial instance to acquire the hyperparameter values required
* Fit the model
* Evaluate on the validation and return the result of a desired metric

In [9]:
import optuna
import pandas as pd
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function(optuna_trial):
    
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky', 'euclidean'])
    
    params = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 500),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 100),
                  }
    
    if similarity == "asymmetric":
        params["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 0.15, log=False)
        params["normalize"] = True     

    elif similarity == "tversky":
        params["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        params["normalize"] = True 

    elif similarity == "euclidean":
        params["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        params["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        params["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    MAP = 0
    for i in range(k_fold):
        recommender = ItemKNNCFRecommender(urm_trains[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    
    return MAP

/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 3: Create an object/function that will be called after every call of the objective function to log the results in a dataframe for easier readability

In [10]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/KNN_similarity_search.csv", index = False)

### Step 4: Create an optuna Study and run

In [12]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 500)

[I 2024-11-27 17:13:30,856] A new study created in memory with name: no-name-26ed80dc-ea18-4615-8161-657d3e1f6de0


Similarity column 38121 (100.0%), 5528.75 column/sec. Elapsed time 6.90 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 14.70 sec. Users per second: 2402
Similarity column 38121 (100.0%), 5321.37 column/sec. Elapsed time 7.16 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 14.67 sec. Users per second: 2405
Similarity column 38121 (100.0%), 5556.82 column/sec. Elapsed time 6.86 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 14.41 sec. Users per second: 2449
Similarity column 38121 (100.0%), 5652.62 column/sec. Elapsed time 6.74 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 14.42 sec. Users per second: 2445
Similarity column 38121 (100.0%), 5496.14 column/sec. Elapsed time 6.94 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 14.55 sec. Users per second: 2426
Similarity column 38121 (100.0%), 5619.08 column/sec. Elapsed time 6.78 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 15.08 sec. Users per second: 2343
Similarity column 38121 (100.0%), 5555.46 column/sec. Elapsed ti

[I 2024-11-27 17:17:07,401] Trial 0 finished with value: 0.026694197434272114 and parameters: {'similarity': 'jaccard', 'topK': 155, 'shrink': 61}. Best is trial 0 with value: 0.026694197434272114.


Similarity column 38121 (100.0%), 5523.51 column/sec. Elapsed time 6.90 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 15.39 sec. Users per second: 2295
Similarity column 38121 (100.0%), 5570.75 column/sec. Elapsed time 6.84 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 15.35 sec. Users per second: 2300
Similarity column 38121 (100.0%), 5527.68 column/sec. Elapsed time 6.90 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 15.34 sec. Users per second: 2300
Similarity column 38121 (100.0%), 5460.54 column/sec. Elapsed time 6.98 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 15.38 sec. Users per second: 2294
Similarity column 38121 (100.0%), 5557.37 column/sec. Elapsed time 6.86 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 15.42 sec. Users per second: 2290
Similarity column 38121 (100.0%), 5405.97 column/sec. Elapsed time 7.05 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 15.11 sec. Users per second: 2337
Similarity column 38121 (100.0%), 5629.85 column/sec. Elapsed ti

[I 2024-11-27 17:20:50,843] Trial 1 finished with value: 0.02629863696172157 and parameters: {'similarity': 'jaccard', 'topK': 192, 'shrink': 79}. Best is trial 0 with value: 0.026694197434272114.


...

Similarity column 38121 (100.0%), 5239.70 column/sec. Elapsed time 7.28 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 11.28 sec. Users per second: 3132
Similarity column 38121 (100.0%), 5196.05 column/sec. Elapsed time 7.34 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 10.92 sec. Users per second: 3232
Similarity column 38121 (100.0%), 5289.38 column/sec. Elapsed time 7.21 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 10.87 sec. Users per second: 3248
Similarity column 38121 (100.0%), 5227.38 column/sec. Elapsed time 7.29 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 11.60 sec. Users per second: 3041
Similarity column 38121 (100.0%), 5486.16 column/sec. Elapsed time 6.95 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 11.23 sec. Users per second: 3145
Similarity column 38121 (100.0%), 5151.57 column/sec. Elapsed time 7.40 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 11.52 sec. Users per second: 3067
Similarity column 38121 (100.0%), 4998.09 column/sec. Elapsed ti

[I 2024-11-27 20:56:24,873] Trial 51 finished with value: 0.02925204846285377 and parameters: {'similarity': 'asymmetric', 'topK': 25, 'shrink': 5, 'asymmetric_alpha': 0.14817340240454221}. Best is trial 43 with value: 0.030736762228581672.


Similarity column 38121 (100.0%), 5337.41 column/sec. Elapsed time 7.14 sec


[W 2024-11-27 20:56:42,024] Trial 52 failed with parameters: {'similarity': 'asymmetric', 'topK': 6, 'shrink': 25, 'asymmetric_alpha': 0.12605589777718657} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-9-bab864f4fcbe>", line 33, in objective_function
    result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Evaluation/Evaluator.py", line 484, in _run_evaluation_on_selected_users
    results_dict = self._compute_metrics_on_recommendation_list(test_user_batch_array = test_us

KeyboardInterrupt: 

# First Optuna Analysis
We can infer from the first iteration (~50) that *euclidian* consistently has worse results and that we tend to get better performance with small value of K and shrink with *asymmetric* and *tversky* similarity functions, specifically with a **MAP** of around 0.30. Therefore, to better improve the hyperparameter research I will repeat the process with a smaller hyperparameter feature space, hoping that it will speed up the convergence. For this reason I killed the above cell.

In [13]:
import optuna
import pandas as pd
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function(optuna_trial):
    
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky'])
    
    params = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 80),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 50),
                  }
    
    if similarity == "asymmetric":
        params["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 0.15, log=False)
        params["normalize"] = True     

    elif similarity == "tversky":
        params["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        params["normalize"] = True 

    MAP = 0
    for i in range(k_fold):
        recommender = ItemKNNCFRecommender(urm_trains[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    
    return MAP

In [14]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/KNN_similarity_search_smaller_hyper.csv", index = False)

In [15]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 200)

[I 2024-11-27 21:02:46,482] A new study created in memory with name: no-name-0b4b2fa0-4c37-4ed7-a270-bf13725d6027


Similarity column 38121 (100.0%), 5645.67 column/sec. Elapsed time 6.75 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 12.15 sec. Users per second: 2907
Similarity column 38121 (100.0%), 5674.18 column/sec. Elapsed time 6.72 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 11.83 sec. Users per second: 2982
Similarity column 38121 (100.0%), 5701.23 column/sec. Elapsed time 6.69 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 11.85 sec. Users per second: 2979
Similarity column 38121 (100.0%), 5758.46 column/sec. Elapsed time 6.62 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 11.79 sec. Users per second: 2992
Similarity column 38121 (100.0%), 5717.29 column/sec. Elapsed time 6.67 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 11.98 sec. Users per second: 2947
Similarity column 38121 (100.0%), 5634.16 column/sec. Elapsed time 6.77 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 11.89 sec. Users per second: 2971
Similarity column 38121 (100.0%), 5741.91 column/sec. Elapsed ti

[I 2024-11-27 21:05:53,445] Trial 0 finished with value: 0.02725874946486771 and parameters: {'similarity': 'tversky', 'topK': 41, 'shrink': 24, 'tversky_alpha': 1.0490696185422574, 'tversky_beta': 0.5956663450958362}. Best is trial 0 with value: 0.02725874946486771.


Similarity column 38121 (100.0%), 5633.25 column/sec. Elapsed time 6.77 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 10.71 sec. Users per second: 3296
Similarity column 38121 (100.0%), 5697.78 column/sec. Elapsed time 6.69 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 10.70 sec. Users per second: 3298
Similarity column 38121 (100.0%), 5666.72 column/sec. Elapsed time 6.73 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 10.73 sec. Users per second: 3289
Similarity column 38121 (100.0%), 5663.16 column/sec. Elapsed time 6.73 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 10.70 sec. Users per second: 3296
Similarity column 38121 (100.0%), 5705.64 column/sec. Elapsed time 6.68 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 10.68 sec. Users per second: 3305
Similarity column 38121 (100.0%), 5559.68 column/sec. Elapsed time 6.86 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 10.71 sec. Users per second: 3298
Similarity column 38121 (100.0%), 5663.92 column/sec. Elapsed ti

[I 2024-11-27 21:08:48,210] Trial 1 finished with value: 0.028234284284982352 and parameters: {'similarity': 'cosine', 'topK': 9, 'shrink': 43}. Best is trial 1 with value: 0.028234284284982352.


...

[I 2024-11-28 06:33:32,521] Trial 197 finished with value: 0.03077653029029358 and parameters: {'similarity': 'asymmetric', 'topK': 6, 'shrink': 16, 'asymmetric_alpha': 0.034408229683770726}. Best is trial 155 with value: 0.03078142287004788.


Similarity column 38121 (100.0%), 5970.05 column/sec. Elapsed time 6.39 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 10.62 sec. Users per second: 3325
Similarity column 38121 (100.0%), 5976.13 column/sec. Elapsed time 6.38 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 10.66 sec. Users per second: 3311
Similarity column 38121 (100.0%), 5963.11 column/sec. Elapsed time 6.39 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 10.58 sec. Users per second: 3335
Similarity column 38121 (100.0%), 5971.60 column/sec. Elapsed time 6.38 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 10.56 sec. Users per second: 3340
Similarity column 38121 (100.0%), 5950.51 column/sec. Elapsed time 6.41 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 10.61 sec. Users per second: 3327
Similarity column 38121 (100.0%), 5975.61 column/sec. Elapsed time 6.38 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 10.61 sec. Users per second: 3330
Similarity column 38121 (100.0%), 5967.65 column/sec. Elapsed ti

[I 2024-11-28 06:36:23,701] Trial 198 finished with value: 0.027980267370226557 and parameters: {'similarity': 'tversky', 'topK': 9, 'shrink': 15, 'tversky_alpha': 0.13108245931890683, 'tversky_beta': 0.16272028306527697}. Best is trial 155 with value: 0.03078142287004788.


Similarity column 38121 (100.0%), 5668.67 column/sec. Elapsed time 6.72 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 10.33 sec. Users per second: 3418
Similarity column 38121 (100.0%), 5679.65 column/sec. Elapsed time 6.71 sec
EvaluatorHoldout: Processed 35289 (100.0%) in 10.36 sec. Users per second: 3407
Similarity column 38121 (100.0%), 5679.61 column/sec. Elapsed time 6.71 sec
EvaluatorHoldout: Processed 35295 (100.0%) in 10.32 sec. Users per second: 3419
Similarity column 38121 (100.0%), 5694.32 column/sec. Elapsed time 6.69 sec
EvaluatorHoldout: Processed 35269 (100.0%) in 10.28 sec. Users per second: 3429
Similarity column 38121 (100.0%), 5633.79 column/sec. Elapsed time 6.77 sec
EvaluatorHoldout: Processed 35311 (100.0%) in 10.31 sec. Users per second: 3427
Similarity column 38121 (100.0%), 5689.72 column/sec. Elapsed time 6.70 sec
EvaluatorHoldout: Processed 35323 (100.0%) in 10.31 sec. Users per second: 3428
Similarity column 38121 (100.0%), 5628.78 column/sec. Elapsed ti

[I 2024-11-28 06:39:14,413] Trial 199 finished with value: 0.03077462836249707 and parameters: {'similarity': 'asymmetric', 'topK': 6, 'shrink': 16, 'asymmetric_alpha': 0.03420883590351088}. Best is trial 155 with value: 0.03078142287004788.


In [19]:
best_params = optuna_study.best_params
print(best_params)

{'similarity': 'asymmetric', 'topK': 6, 'shrink': 16, 'asymmetric_alpha': 0.024628530649850728}


## Test value
Let's see how it comparsed with the test set

In [21]:
urm_train_validations = []
for i in range(k_fold):
    urm_train_validations.append(sps.vstack([urm_trains[i], urm_validations[i]]))

best_MAP_test = 0

for i in range(k_fold):
    recommender = ItemKNNCFRecommender(urm_train_validations[i])
    recommender.fit(**best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

ItemKNNCFRecommender: URM Detected 425 ( 0.6%) users with no interactions.
Similarity column 38121 (100.0%), 5330.30 column/sec. Elapsed time 7.15 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.84 sec. Users per second: 3281
ItemKNNCFRecommender: URM Detected 447 ( 0.6%) users with no interactions.
Similarity column 38121 (100.0%), 5390.94 column/sec. Elapsed time 7.07 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 10.75 sec. Users per second: 3311
ItemKNNCFRecommender: URM Detected 441 ( 0.6%) users with no interactions.
Similarity column 38121 (100.0%), 5332.37 column/sec. Elapsed time 7.15 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.74 sec. Users per second: 3313
ItemKNNCFRecommender: URM Detected 467 ( 0.7%) users with no interactions.
Similarity column 38121 (100.0%), 5385.65 column/sec. Elapsed time 7.08 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 10.78 sec. Users per second: 3299
ItemKNNCFRecommender: URM Detected 425 ( 0.6%) users with no interactions.
S

0.04021931143181421

# Let's try predict the best item for users
Let's see which items recommend with the current configuration. Then it may make sense to retray optuna with an even smaller hyperparameter with 80% of the whole dataset to see which results it provides.

In [22]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


In [ ]:
recommender = ItemKNNCFRecommender(URM_all)
recommender.fit(**best_params)

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv("submissions/knn_Sim:{}_K:{}_Shr:{}_AsymAlpha{}.csv".format(
    best_params["similarity"], 
    best_params["topK"], 
    best_params["shrink"], 
    best_params["asymmetric_alpha"]), 
    index=False)

# We can further improve the hyperparameters repeating the Optuna process by including the dataset

In [34]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()
urm_trains = []
urm_validations = []
urm_train_validations = []
urm_test = []
# Defining evaluators
evaluator_validation = []
evaluator_test = []

for i in range(k_fold):
    # Splitting data into 64% train, 16% validation, 20% test
    URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=seeds[i])
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8, seed=seeds[i])
    
    urm_trains.append(URM_train)
    urm_validations.append(URM_validation)
    urm_train_validations.append(URM_train_validation)
    urm_test.append(URM_test)
    
    evaluator_validation.append(EvaluatorHoldout(URM_validation, cutoff_list=[10]))
    evaluator_test.append(EvaluatorHoldout(URM_test, cutoff_list=[10]))

EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 447 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 155 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 441 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 467 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 169 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 413 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 154 ( 0.4%) Users that have less than 

In [36]:
import optuna
import pandas as pd
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

def objective_function(optuna_trial):
    
    similarity = optuna_trial.suggest_categorical("similarity", ['cosine', 'dice', 'jaccard', 'asymmetric', 'tversky'])
    
    params = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 5, 20),
                   "shrink": optuna_trial.suggest_int("shrink", 0, 30),
                  }
    
    if similarity == "asymmetric":
        params["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 0.15, log=False)
        params["normalize"] = True     

    elif similarity == "tversky":
        params["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0, 2, log=False)
        params["normalize"] = True 

    MAP = 0
    for i in range(k_fold):
        recommender = ItemKNNCFRecommender(urm_train_validations[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

        MAP += result_dict.loc[10]["MAP"]
    
    MAP /= k_fold
    
    return MAP

In [37]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/KNN_similarity_final_v2.csv", index = False)

In [38]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100)

[I 2024-11-28 15:34:52,319] A new study created in memory with name: no-name-ddc324a3-7660-4a33-b011-79c8a36bd6cf


Similarity column 38121 (100.0%), 5020.15 column/sec. Elapsed time 7.59 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 11.21 sec. Users per second: 3173
Similarity column 38121 (100.0%), 5026.71 column/sec. Elapsed time 7.58 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 11.23 sec. Users per second: 3169
Similarity column 38121 (100.0%), 4994.93 column/sec. Elapsed time 7.63 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 11.14 sec. Users per second: 3193
Similarity column 38121 (100.0%), 5053.22 column/sec. Elapsed time 7.54 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 11.16 sec. Users per second: 3186
Similarity column 38121 (100.0%), 5050.17 column/sec. Elapsed time 7.55 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 11.11 sec. Users per second: 3201
Similarity column 38121 (100.0%), 5044.62 column/sec. Elapsed time 7.56 sec
EvaluatorHoldout: Processed 35582 (100.0%) in 11.21 sec. Users per second: 3176
Similarity column 38121 (100.0%), 5048.16 column/sec. Elapsed ti

[I 2024-11-28 15:38:01,076] Trial 0 finished with value: 0.049956190873710456 and parameters: {'similarity': 'tversky', 'topK': 19, 'shrink': 22, 'tversky_alpha': 0.9545980511376628, 'tversky_beta': 1.1748421355890462}. Best is trial 0 with value: 0.049956190873710456.


Similarity column 38121 (100.0%), 4942.10 column/sec. Elapsed time 7.71 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.54 sec. Users per second: 3374
Similarity column 38121 (100.0%), 5105.84 column/sec. Elapsed time 7.47 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 10.74 sec. Users per second: 3313
Similarity column 38121 (100.0%), 4913.01 column/sec. Elapsed time 7.76 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.87 sec. Users per second: 3273
Similarity column 38121 (100.0%), 5137.84 column/sec. Elapsed time 7.42 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 10.69 sec. Users per second: 3328
Similarity column 38121 (100.0%), 4880.95 column/sec. Elapsed time 7.81 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 10.98 sec. Users per second: 3240
Similarity column 38121 (100.0%), 4959.21 column/sec. Elapsed time 7.69 sec
EvaluatorHoldout: Processed 35582 (100.0%) in 10.94 sec. Users per second: 3253
Similarity column 38121 (100.0%), 4880.25 column/sec. Elapsed ti

[I 2024-11-28 15:41:06,630] Trial 1 finished with value: 0.05018479770296993 and parameters: {'similarity': 'jaccard', 'topK': 13, 'shrink': 18}. Best is trial 1 with value: 0.05018479770296993.


...

[I 2024-11-28 20:37:24,221] Trial 97 finished with value: 0.05386923674270515 and parameters: {'similarity': 'asymmetric', 'topK': 5, 'shrink': 30, 'asymmetric_alpha': 0.08142309494604869}. Best is trial 74 with value: 0.05475680745670376.


Similarity column 38121 (100.0%), 4767.06 column/sec. Elapsed time 8.00 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.94 sec. Users per second: 3251
Similarity column 38121 (100.0%), 4726.23 column/sec. Elapsed time 8.07 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 10.91 sec. Users per second: 3261
Similarity column 38121 (100.0%), 4916.35 column/sec. Elapsed time 7.75 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 11.00 sec. Users per second: 3233
Similarity column 38121 (100.0%), 4759.98 column/sec. Elapsed time 8.01 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 10.92 sec. Users per second: 3258
Similarity column 38121 (100.0%), 4769.28 column/sec. Elapsed time 7.99 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 10.99 sec. Users per second: 3237
Similarity column 38121 (100.0%), 4770.37 column/sec. Elapsed time 7.99 sec
EvaluatorHoldout: Processed 35582 (100.0%) in 10.97 sec. Users per second: 3243
Similarity column 38121 (100.0%), 4722.92 column/sec. Elapsed ti

[I 2024-11-28 20:40:34,378] Trial 98 finished with value: 0.054472943037449505 and parameters: {'similarity': 'asymmetric', 'topK': 6, 'shrink': 11, 'asymmetric_alpha': 0.04021416561504586}. Best is trial 74 with value: 0.05475680745670376.


Similarity column 38121 (100.0%), 4854.54 column/sec. Elapsed time 7.85 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.92 sec. Users per second: 3258
Similarity column 38121 (100.0%), 5106.73 column/sec. Elapsed time 7.46 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 10.79 sec. Users per second: 3297
Similarity column 38121 (100.0%), 5061.69 column/sec. Elapsed time 7.53 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 10.56 sec. Users per second: 3367
Similarity column 38121 (100.0%), 5124.68 column/sec. Elapsed time 7.44 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 10.81 sec. Users per second: 3290
Similarity column 38121 (100.0%), 5122.73 column/sec. Elapsed time 7.44 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 10.66 sec. Users per second: 3336
Similarity column 38121 (100.0%), 5087.12 column/sec. Elapsed time 7.49 sec
EvaluatorHoldout: Processed 35582 (100.0%) in 10.68 sec. Users per second: 3331
Similarity column 38121 (100.0%), 5123.18 column/sec. Elapsed ti

[I 2024-11-28 20:43:37,980] Trial 99 finished with value: 0.05183022613290113 and parameters: {'similarity': 'tversky', 'topK': 5, 'shrink': 19, 'tversky_alpha': 0.5270943712746126, 'tversky_beta': 0.6017617701560966}. Best is trial 74 with value: 0.05475680745670376.


In [40]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()

In [41]:
best_hyperparams

{'result': 0.05475680745670376,
 'similarity': 'asymmetric',
 'topK': 5.0,
 'shrink': 11.0,
 'tversky_alpha': nan,
 'tversky_beta': nan,
 'asymmetric_alpha': 0.14401030086806713}